In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/bike-sharing-demand/test.csv
/kaggle/input/bike-sharing-demand/sampleSubmission.csv
/kaggle/input/bike-sharing-demand/train.csv


In [2]:
train = pd.read_csv("/kaggle/input/bike-sharing-demand/train.csv")
train.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [3]:
test = pd.read_csv("/kaggle/input/bike-sharing-demand/test.csv")
test.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014


In [4]:
#먼저 모델링
#데이터 갯수가 작으면 cross validation/ , 많으면 훈련 / 평가 셋으로 나누어서 진행(test set의 분포확인 -> train set에서 평가 set을 뽑을 때 모델을 자체적으로 검증하기 위해서 최대한 테스트 셋이랑 비슷한 set을 뽑아야함)
#통계량, 시각화 등을 통해 변수가 모델에 영향을 미치는 지 
# 데이터 셋에 맞는 모델이 존재 - category형(문자형) 변수가 많을수록 tree model에 잘 동작(학습을 잘함, 점수도 잘나옴)/ 데이터 의 비선형성이 많을 수록 트리모델에 동작
# randomforest , catboost, lgbm ,xgboost (randomforest 가장 기본 모델, 모델링도 쉽고, 데이터 전처리가 필요없음; )
# 하이퍼파라미터 중요..;; 연도,월,일 과같은 변수 추가했을 때 모델 점수가 좋아질 줄 알았는데 떨어짐..;;
# 데이터 셋이 복잡해질수록 모델이 학습을 잘 못할 수 있다 --> 하이퍼파라미터 옵션 지정 필요 (학습량 늘려주기) - 하이퍼파라미터 설정으로 모델의 점수가 올라감 , 당장 성능이 좋아지지 않아도 나중에 좋아질 수 있음 

In [5]:
train.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [6]:
#train["hour"] = train.datetime.str[11:13]
train["datetime"] = train["datetime"].astype("datetime64")

In [7]:
test["datetime"] = test["datetime"].astype("datetime64")

In [8]:
train["hour"] = train["datetime"].dt.hour
test["hour"] = test["datetime"].dt.hour

In [9]:
#train["year"]= train["datetime"].dt.year
#test["year"] = test["datetime"].dt.year

In [10]:
y= train["count"]
y.head()

0    16
1    40
2    32
3    13
4     1
Name: count, dtype: int64

In [11]:
x=train.drop(["datetime","casual","registered","count"],1)
x.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,hour
0,1,0,0,1,9.84,14.395,81,0.0,0
1,1,0,0,1,9.02,13.635,80,0.0,1
2,1,0,0,1,9.02,13.635,80,0.0,2
3,1,0,0,1,9.84,14.395,75,0.0,3
4,1,0,0,1,9.84,14.395,75,0.0,4


In [12]:
x_test = test.drop("datetime",1)
x_test.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,hour
0,1,0,1,1,10.66,11.365,56,26.0027,0
1,1,0,1,1,10.66,13.635,56,0.0000,1
2,1,0,1,1,10.66,13.635,56,0.0000,2
3,1,0,1,1,10.66,12.880,56,11.0014,3
4,1,0,1,1,10.66,12.880,56,11.0014,4


In [13]:
train.dtypes

datetime      datetime64[ns]
season                 int64
holiday                int64
workingday             int64
weather                int64
temp                 float64
atemp                float64
humidity               int64
windspeed            float64
casual                 int64
registered             int64
count                  int64
hour                   int64
dtype: object

In [14]:
from sklearn.ensemble import RandomForestRegressor

In [15]:
rf = RandomForestRegressor()

In [16]:
rf.fit(x,y)

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [17]:
rf.feature_importances_

array([0.03830756, 0.00351923, 0.07683151, 0.02043561, 0.09916131,
       0.05859294, 0.06718992, 0.03761037, 0.59835155])

In [18]:
result = rf.predict(x_test)

In [19]:
sub = pd.read_csv("/kaggle/input/bike-sharing-demand/sampleSubmission.csv")
sub.head()

,datetime,count
0,2011-01-20 00:00:00,0
1,2011-01-20 01:00:00,0
2,2011-01-20 02:00:00,0
3,2011-01-20 03:00:00,0
4,2011-01-20 04:00:00,0


In [20]:
sub["count"] = result

In [21]:
sub.head()

,datetime,count
0,2011-01-20 00:00:00,11.6
1,2011-01-20 01:00:00,5.6
2,2011-01-20 02:00:00,3.7
3,2011-01-20 03:00:00,3.1
4,2011-01-20 04:00:00,2.0


In [22]:
sub.to_csv("submission.csv",index=False)
#index =True : 0,1,2,3,4...(index)가 새로운 컬럼으로 나오게 됨. 